## IMPORT

In [1]:
import pandas as pd 
import numpy as np
from pytrends.request import TrendReq
from pytrends import dailydata
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [2]:
pip install pytrends

Note: you may need to restart the kernel to use updated packages.


## 각 자산별 DATA

In [3]:
Ndata = pd.read_csv("nas.csv")
Gdata = pd.read_csv("gold.csv")
Odata = pd.read_csv("oildpb.csv")
Bdata = pd.read_csv("bitcoin.csv")
Bndata = pd.read_csv("bnb.csv")
Rdata = pd.read_csv("xrp.csv")
Tdata = pd.read_csv("tron.csv")
Sdata = pd.read_csv("sbi.csv")
Ddata = pd.read_csv("doge.csv")
Edata = pd.read_csv("eth.csv")
Adata = pd.read_csv("ada.csv")

In [4]:
oil = Odata.set_index("DATE")
oil.columns = ['OILPRICE']
oil.index = pd.to_datetime(oil.index)
oil = oil.sort_index()

In [5]:
bnb = pd.DataFrame(Bndata.iloc[:,0:2]).set_index("DATE")
bnb.columns = ['BNB']
bnb.index = pd.to_datetime(bnb.index)
bnb = bnb.sort_index()
pbnb = bnb.pct_change().dropna()


In [61]:
xrp = pd.DataFrame(Rdata.iloc[:,0:2]).set_index("DATE")
xrp.columns = ["XRP"]
xrp.index = pd.to_datetime(xrp.index)
xrp = xrp.sort_index()
pxrp = xrp.pct_change().dropna()

In [62]:
btc = pd.DataFrame(Bdata.iloc[:,0:2]).set_index("DATE")
btc.columns = ["BTC"]
btc.index = pd.to_datetime(btc.index)
btc = btc.sort_index()
pbtc = btc.pct_change().dropna()

In [63]:
tron = pd.DataFrame(Tdata.iloc[:,0:2]).set_index("DATE")
tron.columns = ["TRON"]
tron.index = pd.to_datetime(tron.index)
tron = tron.sort_index()
ptron = tron.pct_change().dropna()

In [64]:
sbi = pd.DataFrame(Sdata.iloc[:,0:2]).set_index("Date")
sbi.columns = ["SHIB"]
sbi.index = pd.to_datetime(sbi.index)
sbi = sbi.sort_index()
psbi = sbi.pct_change().dropna()

In [65]:
doge = pd.DataFrame(Ddata.iloc[:,0:2]).set_index("Date")
doge.columns = ["DOGE"]
doge.index = pd.to_datetime(doge.index)
doge = doge.sort_index()
pdoge = doge.pct_change().dropna()

In [66]:
eth = pd.DataFrame(Edata.iloc[:,0:2]).set_index("Date")
eth.columns = ["ETH"]
eth.index = pd.to_datetime(eth.index)
eth = eth.sort_index()
peth = eth.pct_change().dropna()

In [67]:
ada = pd.DataFrame(Adata.iloc[:,0:2]).set_index("Date")
ada.columns = ["ADA"]
ada.index = pd.to_datetime(ada.index)
ada = ada.sort_index()
pada = ada.pct_change().dropna()

In [68]:
gold = pd.DataFrame(Gdata.iloc[:,0:2]).set_index("DATE")
gold.columns = ["GOLD"]
gold.index = pd.to_datetime(gold.index)
gold = gold.sort_index()
pgold = gold.pct_change().dropna()

In [69]:
nas = pd.DataFrame(Ndata.iloc[:,0:2]).set_index("DATE")
nas.columns = ["NASDAQ"]
nas.index = pd.to_datetime(nas.index)
nas = nas.sort_index()
pnas = nas.pct_change().dropna()

In [70]:
idx = oil[oil["OILPRICE"] == "."].index
oil.drop(idx, inplace = True)
foil = oil.astype(float)
poil = foil.pct_change().dropna()

In [71]:
df = pd.concat([pnas, pgold, poil, pbtc, peth, pbnb, pxrp, pada, ptron, pdoge, psbi], join='outer', axis = 1)

## 

## 비트코인과 나스닥의 상관관계
 - t-value는 유의함을 보여주지만 상관관계는 0.41정도로 낮게 나타나기 때문에 공선성문제가 생기지 않을 것으로 판단하여 이후 OLS에서 둘 다 독립변수로 사용하였음

In [72]:
df_n = df.loc[:, ["BTC", "NASDAQ"]].dropna()
y_n = df_n.loc[:, ["BTC"]].dropna()
x_n = df_n.loc[:, ["NASDAQ"]].dropna()
x_n = sm.add_constant(x_n)
df.loc[:, ["BTC", "NASDAQ"]].corr()

/Users/boom/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


,BTC,NASDAQ
BTC,1.000000,0.411842
NASDAQ,0.411842,1.000000


In [73]:
nas_model = sm.OLS(y_n, x_n).fit().summary()
nas_model

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    BTC   R-squared:                       0.170
Model:                            OLS   Adj. R-squared:                  0.168
Method:                 Least Squares   F-statistic:                     123.4
Date:                Sun, 05 Jun 2022   Prob (F-statistic):           3.28e-26
Time:                        19:47:51   Log-Likelihood:                 1085.1
No. Observations:                 606   AIC:                            -2166.
Df Residuals:                     604   BIC:                            -2157.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0027      0.002      1.633      0.103      -0.001       0.006
NASDAQ         0.9898      0.089     11.107      0.000       0.815       1.165
==============================================================================
Omnibus:                       97.473   Durbin-Watson:                   2.104
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              906.935
Skew:                          -0.364   Prob(JB):                    1.15e-197
Kurtosis:                       8.949   Cond. No.                         54.2
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## 각 코인별 데일리 트렌드 지수

In [74]:
daily_btc = pd.read_csv("daily_btc.csv")
daily_eth = pd.read_csv("daily_eth.csv")
daily_bnb = pd.read_csv("daily_bnb.csv")
daily_xrp = pd.read_csv("daily_xrp.csv")
daily_ada = pd.read_csv("daily_ada.csv")
daily_trn = pd.read_csv("daily_trn.csv")
daily_dog = pd.read_csv("daily_dog.csv")
daily_shb = pd.read_csv("daily_shb.csv")

In [75]:
dbtc = pd.DataFrame(daily_btc.iloc[:,1:3]).set_index("date")
dbtc.columns = ["daily_BTC"]
dbtc.index = pd.to_datetime(dbtc.index)
dbtc = dbtc.sort_index()

deth = pd.DataFrame(daily_eth.iloc[:,1:3]).set_index("date")
deth.columns = ["daily_ETH"]
deth.index = pd.to_datetime(deth.index)
deth = deth.sort_index()

dbnb = pd.DataFrame(daily_bnb.iloc[:,1:3]).set_index("date")
dbnb.columns = ["daily_BNB"]
dbnb.index = pd.to_datetime(dbnb.index)
dbnb = dbnb.sort_index()

dxrp = pd.DataFrame(daily_xrp.iloc[:,1:3]).set_index("date")
dxrp.columns = ["daily_XRP"]
dxrp.index = pd.to_datetime(dxrp.index)
dxrp = dxrp.sort_index()

dada = pd.DataFrame(daily_ada.iloc[:,1:3]).set_index("date")
dada.columns = ["daily_ADA"]
dada.index = pd.to_datetime(dada.index)
dada = dada.sort_index()

dtrn = pd.DataFrame(daily_trn.iloc[:,1:3]).set_index("date")
dtrn.columns = ["daily_TRON"]
dtrn.index = pd.to_datetime(dtrn.index)
dtrn = dtrn.sort_index()

ddog = pd.DataFrame(daily_dog.iloc[:,1:3]).set_index("date")
ddog.columns = ["daily_DOGE"]
ddog.index = pd.to_datetime(ddog.index)
ddog = ddog.sort_index()

dsbi = pd.DataFrame(daily_shb.iloc[:,1:3]).set_index("date")
dsbi.columns = ["daily_SHIB"]
dsbi.index = pd.to_datetime(dsbi.index)
dsbi = dsbi.sort_index()

## 더미변수 만들기

In [76]:
df_eth = df.loc[:, ["ETH", "NASDAQ", "GOLD", "OILPRICE", "BTC"]].dropna()
df_eth = pd.concat([df_eth, deth], join = 'inner', axis = 1).dropna()
df_bnb = df.loc[:, ["BNB", "NASDAQ", "GOLD", "OILPRICE", "BTC"]].dropna()
df_bnb = pd.concat([df_bnb, dbnb], join = 'inner', axis = 1).dropna()
df_xrp = df.loc[:, ["XRP", "NASDAQ", "GOLD", "OILPRICE", "BTC"]].dropna()
df_xrp = pd.concat([df_xrp, dxrp], join = 'inner', axis = 1).dropna()
df_ada = df.loc[:, ["ADA", "NASDAQ", "GOLD", "OILPRICE", "BTC"]].dropna()
df_ada = pd.concat([df_ada, dada], join = 'inner', axis = 1).dropna()
df_trn = df.loc[:, ["TRON", "NASDAQ", "GOLD", "OILPRICE", "BTC"]].dropna()
df_trn = pd.concat([df_trn, dtrn], join = 'inner', axis = 1).dropna()
df_dog = df.loc[:, ["DOGE", "NASDAQ", "GOLD", "OILPRICE", "BTC"]].dropna()
df_dog = pd.concat([df_dog, ddog], join = 'inner', axis = 1).dropna()
df_sbi = df.loc[:, ["SHIB", "NASDAQ", "GOLD", "OILPRICE", "BTC"]].dropna()
df_sbi = pd.concat([df_sbi, dsbi], join = 'inner', axis = 1).dropna()

In [78]:
eth_d = df_eth["ETH"].apply(lambda x : True if x >= 0 else False)
eth_d = pd.get_dummies(pd.DataFrame(eth_d), columns = ["ETH"])
dd_eth = pd.concat([eth_d,deth], join='inner', axis=1).dropna()
dd_eth = dd_eth.loc[:,['ETH_False', 'ETH_True']].mul(dd_eth.loc[:,"daily_ETH"], axis = 0)

bnb_d = df_bnb["BNB"].apply(lambda x : True if x >= 0 else False)
bnb_d = pd.get_dummies(pd.DataFrame(bnb_d), columns = ["BNB"])
dd_bnb = pd.concat([bnb_d,dbnb], join='inner', axis=1).dropna()
dd_bnb = dd_bnb.loc[:,['BNB_False', 'BNB_True']].mul(dd_bnb.loc[:,"daily_BNB"], axis = 0)

xrp_d = df_xrp["XRP"].apply(lambda x : True if x >= 0 else False)
xrp_d = pd.get_dummies(pd.DataFrame(xrp_d), columns = ["XRP"])
dd_xrp = pd.concat([xrp_d,dxrp], join='inner', axis=1).dropna()
dd_xrp = dd_xrp.loc[:,['XRP_False', 'XRP_True']].mul(dd_xrp.loc[:,"daily_XRP"], axis = 0)

ada_d = df_ada["ADA"].apply(lambda x : True if x >= 0 else False)
ada_d = pd.get_dummies(pd.DataFrame(ada_d), columns = ["ADA"])
dd_ada = pd.concat([ada_d,dada], join='inner', axis=1).dropna()
dd_ada = dd_ada.loc[:,['ADA_False', 'ADA_True']].mul(dd_ada.loc[:,"daily_ADA"], axis = 0)

trn_d = df_trn["TRON"].apply(lambda x : True if x >= 0 else False)
trn_d = pd.get_dummies(pd.DataFrame(trn_d), columns = ["TRON"])
dd_trn = pd.concat([trn_d,dtrn], join='inner', axis=1).dropna()
dd_trn = dd_trn.loc[:,['TRON_False', 'TRON_True']].mul(dd_trn.loc[:,"daily_TRON"], axis = 0)

dog_d = df_dog["DOGE"].apply(lambda x : True if x >= 0 else False)
dog_d = pd.get_dummies(pd.DataFrame(dog_d), columns = ["DOGE"])
dd_dog = pd.concat([dog_d,ddog], join='inner', axis=1).dropna()
dd_dog = dd_dog.loc[:,['DOGE_False', 'DOGE_True']].mul(dd_dog.loc[:,"daily_DOGE"], axis = 0)

sbi_d = df_sbi["SHIB"].apply(lambda x : True if x >= 0 else False)
sbi_d = pd.get_dummies(pd.DataFrame(sbi_d), columns = ["SHIB"])
dd_sbi = pd.concat([sbi_d,dsbi], join='inner', axis=1).dropna()
dd_sbi = dd_sbi.loc[:,['SHIB_False', 'SHIB_True']].mul(dd_sbi.loc[:,"daily_SHIB"], axis = 0)

In [87]:
df_e = pd.concat([df_eth, dd_eth], join='inner', axis=1)
df_b = pd.concat([df_bnb, dd_bnb], join='inner', axis=1)
df_x = pd.concat([df_xrp, dd_xrp], join='inner', axis=1)
df_a = pd.concat([df_ada, dd_ada], join='inner', axis=1)
df_t = pd.concat([df_trn, dd_trn], join='inner', axis=1)
df_d = pd.concat([df_dog, dd_dog], join='inner', axis=1)
df_s = pd.concat([df_sbi, dd_sbi], join='inner', axis=1)

## 각각 OLS에 대하여 독립변수, 종속변수 설정

In [81]:
y_e = df_e.loc[:, "ETH"].dropna()
x_e = df_e.loc[:, ["NASDAQ", "GOLD", "OILPRICE", "BTC", "ETH_False", "ETH_True"]].dropna()
#x_e = df_e.loc[:, ["NASDAQ", "GOLD", "OILPRICE", "BTC", "daily_ETH"]].dropna()
x_e = sm.add_constant(x_e)

y_b = df_b.loc[:, "BNB"].dropna()
x_b = df_b.loc[:, ["NASDAQ", "GOLD", "OILPRICE", "BTC", "BNB_False", "BNB_True"]].dropna()
#x_b = df_b.loc[:, ["NASDAQ", "GOLD", "OILPRICE", "BTC", "daily_BNB"]].dropna()
x_b = sm.add_constant(x_b)

y_x = df_x.loc[:, "XRP"].dropna()
x_x = df_x.loc[:, ["NASDAQ", "GOLD", "OILPRICE", "BTC", "XRP_False", "XRP_True"]].dropna()
#x_x = df_x.loc[:, ["NASDAQ", "GOLD", "OILPRICE", "BTC", "daily_XRP"]].dropna()
x_x = sm.add_constant(x_x)

y_a = df_a.loc[:, "ADA"].dropna()
x_a = df_a.loc[:, ["NASDAQ", "GOLD", "OILPRICE", "BTC", "ADA_False", "ADA_True"]].dropna()
#x_a = df_a.loc[:, ["NASDAQ", "GOLD", "OILPRICE", "BTC", "daily_ADA"]].dropna()
x_a = sm.add_constant(x_a)

y_t = df_t.loc[:, "TRON"].dropna()
x_t = df_t.loc[:, ["NASDAQ", "GOLD", "OILPRICE", "BTC", "TRON_False", "TRON_True"]].dropna()
#x_t = df_t.loc[:, ["NASDAQ", "GOLD", "OILPRICE", "BTC", "daily_TRON"]].dropna()
x_t = sm.add_constant(x_t)

y_d = df_d.loc[:, "DOGE"].dropna()
x_d = df_d.loc[:, ["NASDAQ", "GOLD", "OILPRICE", "BTC", "DOGE_False", "DOGE_True"]].dropna()
#x_d = df_d.loc[:, ["NASDAQ", "GOLD", "OILPRICE", "BTC", "daily_DOGE"]].dropna()
x_d = sm.add_constant(x_d)

y_s = df_s.loc[:, "SHIB"].dropna()
x_s = df_s.loc[:, ["NASDAQ", "GOLD", "OILPRICE", "BTC", "SHIB_False", "SHIB_True"]].dropna()
#x_s = df_s.loc[:, ["NASDAQ", "GOLD", "OILPRICE", "BTC", "daily_SHIB"]].dropna()
x_s = sm.add_constant(x_s)

/Users/boom/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


## OLS 결과

In [25]:
eth_model = sm.OLS(y_e, x_e).fit().summary()
bnb_model = sm.OLS(y_b, x_b).fit().summary()
xrp_model = sm.OLS(y_x, x_x).fit().summary()
ada_model = sm.OLS(y_a, x_a).fit().summary()
trn_model = sm.OLS(y_t, x_t).fit().summary()
dog_model = sm.OLS(y_d, x_d).fit().summary()

In [26]:
eth_model

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    ETH   R-squared:                       0.777
Model:                            OLS   Adj. R-squared:                  0.775
Method:                 Least Squares   F-statistic:                     345.2
Date:                Sun, 05 Jun 2022   Prob (F-statistic):          9.07e-190
Time:                        19:44:25   Log-Likelihood:                 1331.1
No. Observations:                 600   AIC:                            -2648.
Df Residuals:                     593   BIC:                            -2618.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0003      0.004     -0.065      0.948      -0.009       0.008
NASDAQ         0.2229      0.065      3.426      0.001       0.095       0.351
GOLD           0.0696      0.095      0.733      0.464      -0.117       0.256
OILPRICE       0.0014      0.008      0.183      0.855      -0.014       0.016
BTC            0.7821      0.032     24.339      0.000       0.719       0.845
ETH_False     -0.0002   6.79e-05     -3.643      0.000      -0.000      -0.000
ETH_True       0.0003   6.46e-05      4.140      0.000       0.000       0.000
==============================================================================
Omnibus:                       81.625   Durbin-Watson:                   2.158
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              684.134
Skew:                           0.237   Prob(JB):                    2.77e-149
Kurtosis:                       8.210   Cond. No.                     4.42e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.42e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [27]:
bnb_model

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    BNB   R-squared:                       0.532
Model:                            OLS   Adj. R-squared:                  0.528
Method:                 Least Squares   F-statistic:                     112.5
Date:                Sun, 05 Jun 2022   Prob (F-statistic):           1.84e-94
Time:                        19:44:25   Log-Likelihood:                 983.73
No. Observations:                 600   AIC:                            -1953.
Df Residuals:                     593   BIC:                            -1923.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0019      0.012      0.167      0.867      -0.021       0.025
NASDAQ         0.1930      0.116      1.662      0.097      -0.035       0.421
GOLD           0.0535      0.169      0.316      0.752      -0.279       0.386
OILPRICE       0.0064      0.014      0.470      0.638      -0.020       0.033
BTC            0.7153      0.056     12.713      0.000       0.605       0.826
BNB_False     -0.0003      0.000     -1.879      0.061      -0.001    1.23e-05
BNB_True       0.0003      0.000      2.038      0.042    1.05e-05       0.001
==============================================================================
Omnibus:                      638.146   Durbin-Watson:                   1.880
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            70179.233
Skew:                           4.598   Prob(JB):                         0.00
Kurtosis:                      55.179   Cond. No.                     5.28e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.28e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [28]:
xrp_model

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    XRP   R-squared:                       0.503
Model:                            OLS   Adj. R-squared:                  0.498
Method:                 Least Squares   F-statistic:                     100.0
Date:                Sun, 05 Jun 2022   Prob (F-statistic):           1.06e-86
Time:                        19:44:25   Log-Likelihood:                 952.31
No. Observations:                 600   AIC:                            -1891.
Df Residuals:                     593   BIC:                            -1860.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0053      0.006      0.917      0.360      -0.006       0.017
NASDAQ         0.0717      0.122      0.586      0.558      -0.169       0.312
GOLD          -0.1706      0.178     -0.957      0.339      -0.521       0.180
OILPRICE      -0.0084      0.014     -0.588      0.557      -0.037       0.020
BTC            0.6704      0.057     11.705      0.000       0.558       0.783
XRP_False     -0.0005      0.000     -5.179      0.000      -0.001      -0.000
XRP_True       0.0003   9.63e-05      3.417      0.001       0.000       0.001
==============================================================================
Omnibus:                      287.179   Durbin-Watson:                   2.033
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             9439.898
Skew:                           1.469   Prob(JB):                         0.00
Kurtosis:                      22.208   Cond. No.                     4.07e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.07e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [29]:
ada_model

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    ADA   R-squared:                       0.632
Model:                            OLS   Adj. R-squared:                  0.629
Method:                 Least Squares   F-statistic:                     170.0
Date:                Sun, 05 Jun 2022   Prob (F-statistic):          2.40e-125
Time:                        19:44:25   Log-Likelihood:                 1065.2
No. Observations:                 600   AIC:                            -2116.
Df Residuals:                     593   BIC:                            -2086.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0221      0.014      1.578      0.115      -0.005       0.050
NASDAQ         0.2472      0.102      2.429      0.015       0.047       0.447
GOLD          -0.0990      0.148     -0.670      0.503      -0.389       0.191
OILPRICE       0.0007      0.012      0.058      0.954      -0.023       0.024
BTC            0.6404      0.047     13.641      0.000       0.548       0.733
ADA_False     -0.0006      0.000     -3.659      0.000      -0.001      -0.000
ADA_True       0.0001      0.000      0.750      0.454      -0.000       0.000
==============================================================================
Omnibus:                      251.306   Durbin-Watson:                   2.103
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2379.563
Skew:                           1.592   Prob(JB):                         0.00
Kurtosis:                      12.222   Cond. No.                     5.45e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.45e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [30]:
trn_model

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   TRON   R-squared:                       0.598
Model:                            OLS   Adj. R-squared:                  0.594
Method:                 Least Squares   F-statistic:                     146.9
Date:                Sun, 05 Jun 2022   Prob (F-statistic):          8.27e-114
Time:                        19:44:25   Log-Likelihood:                 1082.1
No. Observations:                 600   AIC:                            -2150.
Df Residuals:                     593   BIC:                            -2119.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0175      0.009     -1.853      0.064      -0.036       0.001
NASDAQ         0.0147      0.099      0.149      0.881      -0.179       0.208
GOLD           0.0806      0.144      0.560      0.576      -0.202       0.363
OILPRICE      -0.0006      0.012     -0.051      0.959      -0.023       0.022
BTC            0.6473      0.046     14.032      0.000       0.557       0.738
TRON_False    -0.0001      0.000     -0.908      0.364      -0.000       0.000
TRON_True      0.0006      0.000      4.735      0.000       0.000       0.001
==============================================================================
Omnibus:                      282.617   Durbin-Watson:                   2.158
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             5378.774
Skew:                           1.608   Prob(JB):                         0.00
Kurtosis:                      17.311   Cond. No.                     4.47e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.47e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [31]:
dog_model

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   DOGE   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                 -0.003
Method:                 Least Squares   F-statistic:                    0.7489
Date:                Sun, 05 Jun 2022   Prob (F-statistic):              0.610
Time:                        19:44:25   Log-Likelihood:                -1406.5
No. Observations:                 600   AIC:                             2827.
Df Residuals:                     593   BIC:                             2858.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1112      0.264     -0.421      0.674      -0.630       0.407
NASDAQ        -0.1135      6.230     -0.018      0.985     -12.349      12.122
GOLD           3.3306      9.117      0.365      0.715     -14.574      21.235
OILPRICE      -0.0702      0.731     -0.096      0.924      -1.507       1.366
BTC           -1.3139      2.855     -0.460      0.645      -6.920       4.292
DOGE_False     0.0006      0.005      0.137      0.891      -0.009       0.010
DOGE_True      0.0071      0.004      1.602      0.110      -0.002       0.016
==============================================================================
Omnibus:                     1507.769   Durbin-Watson:                   2.035
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          8520391.542
Skew:                          24.033   Prob(JB):                         0.00
Kurtosis:                     584.812   Cond. No.                     4.00e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large,  4e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## 2022~

#### NASDAQ vs BTC

In [32]:
df_n2 = df.loc['2022-01-01':, ["BTC", "NASDAQ"]].dropna()
y_n2 = df_n2.loc[:, ["BTC"]].dropna()
x_n2 = df_n2.loc[:, ["NASDAQ"]].dropna()
x_n2 = sm.add_constant(x_n2)
df_n2.loc[:, ["BTC", "NASDAQ"]].corr()

/Users/boom/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


,BTC,NASDAQ
BTC,1.000000,0.563674
NASDAQ,0.563674,1.000000


In [33]:
nas_model2 = sm.OLS(y_n2, x_n2).fit().summary()
nas_model2

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    BTC   R-squared:                       0.318
Model:                            OLS   Adj. R-squared:                  0.311
Method:                 Least Squares   F-statistic:                     46.57
Date:                Sun, 05 Jun 2022   Prob (F-statistic):           6.90e-10
Time:                        19:44:27   Log-Likelihood:                 205.55
No. Observations:                 102   AIC:                            -407.1
Df Residuals:                     100   BIC:                            -401.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0010      0.003     -0.304      0.762      -0.007       0.005
NASDAQ         1.0291      0.151      6.824      0.000       0.730       1.328
==============================================================================
Omnibus:                       26.461   Durbin-Watson:                   1.876
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               63.422
Skew:                           0.930   Prob(JB):                     1.69e-14
Kurtosis:                       6.386   Cond. No.                         46.8
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

#### ALT vs BTC

In [83]:
df_e2 = df_e.loc['2022-01-01':]
df_b2 = df_b.loc['2022-01-01':]
df_x2 = df_x.loc['2022-01-01':]
df_a2 = df_a.loc['2022-01-01':]
df_t2 = df_t.loc['2022-01-01':]
df_d2 = df_d.loc['2022-01-01':]
df_s2 = df_s.loc['2022-01-01':]

In [85]:
y_e2 = df_e2.loc[:, "ETH"].dropna()
x_e2 = df_e2.loc[:, ["NASDAQ", "GOLD", "OILPRICE", "BTC", "ETH_False", "ETH_True"]].dropna()
x_e2 = sm.add_constant(x_e2)

y_b2 = df_b2.loc[:, "BNB"].dropna()
x_b2 = df_b2.loc[:, ["NASDAQ", "GOLD", "OILPRICE", "BTC", "BNB_False", "BNB_True"]].dropna()
x_b2 = sm.add_constant(x_b2)

y_x2 = df_x2.loc[:, "XRP"].dropna()
x_x2 = df_x2.loc[:, ["NASDAQ", "GOLD", "OILPRICE", "BTC", "XRP_False", "XRP_True"]].dropna()
x_x2 = sm.add_constant(x_x2)

y_a2 = df_a2.loc[:, "ADA"].dropna()
x_a2 = df_a2.loc[:, ["NASDAQ", "GOLD", "OILPRICE", "BTC", "ADA_False", "ADA_True"]].dropna()
x_a2 = sm.add_constant(x_a2)

y_t2 = df_t2.loc[:, "TRON"].dropna()
x_t2 = df_t2.loc[:, ["NASDAQ", "GOLD", "OILPRICE", "BTC", "TRON_False", "TRON_True"]].dropna()
x_t2 = sm.add_constant(x_t2)

y_d2 = df_d2.loc[:, "DOGE"].dropna()
x_d2 = df_d2.loc[:, ["NASDAQ", "GOLD", "OILPRICE", "BTC", "DOGE_False", "DOGE_True"]].dropna()
x_d2 = sm.add_constant(x_d2)

y_s2 = df_s2.loc[:, "SHIB"].dropna()
x_s2 = df_s2.loc[:, ["NASDAQ", "GOLD", "OILPRICE", "BTC", "SHIB_False", "SHIB_True"]].dropna()
x_s2 = sm.add_constant(x_s2)

eth_model2 = sm.OLS(y_e2, x_e2).fit().summary()
bnb_model2 = sm.OLS(y_b2, x_b2).fit().summary()
xrp_model2 = sm.OLS(y_x2, x_x2).fit().summary()
ada_model2 = sm.OLS(y_a2, x_a2).fit().summary()
trn_model2 = sm.OLS(y_t2, x_t2).fit().summary()
dog_model2 = sm.OLS(y_d2, x_d2).fit().summary()
sbi_model2 = sm.OLS(y_s2, x_s2).fit().summary()

/Users/boom/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [37]:
eth_model2

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    ETH   R-squared:                       0.897
Model:                            OLS   Adj. R-squared:                  0.891
Method:                 Least Squares   F-statistic:                     132.5
Date:                Sun, 05 Jun 2022   Prob (F-statistic):           9.48e-43
Time:                        19:44:28   Log-Likelihood:                 273.86
No. Observations:                  98   AIC:                            -533.7
Df Residuals:                      91   BIC:                            -515.6
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0094      0.008      1.243      0.217      -0.006       0.024
NASDAQ         0.3080      0.098      3.148      0.002       0.114       0.502
GOLD           0.0938      0.177      0.529      0.598      -0.258       0.446
OILPRICE       0.0167      0.051      0.327      0.744      -0.084       0.118
BTC            0.7699      0.057     13.431      0.000       0.656       0.884
ETH_False     -0.0003      0.000     -2.801      0.006      -0.000   -8.48e-05
ETH_True    1.648e-05      0.000      0.157      0.875      -0.000       0.000
==============================================================================
Omnibus:                        6.423   Durbin-Watson:                   1.944
Prob(Omnibus):                  0.040   Jarque-Bera (JB):                6.614
Skew:                          -0.412   Prob(JB):                       0.0366
Kurtosis:                       3.971   Cond. No.                     6.27e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.27e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [38]:
bnb_model2

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    BNB   R-squared:                       0.783
Model:                            OLS   Adj. R-squared:                  0.768
Method:                 Least Squares   F-statistic:                     54.58
Date:                Sun, 05 Jun 2022   Prob (F-statistic):           4.85e-28
Time:                        19:44:28   Log-Likelihood:                 238.61
No. Observations:                  98   AIC:                            -463.2
Df Residuals:                      91   BIC:                            -445.1
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0063      0.029      0.215      0.830      -0.052       0.064
NASDAQ         0.1458      0.136      1.073      0.286      -0.124       0.416
GOLD          -0.0014      0.253     -0.005      0.996      -0.504       0.502
OILPRICE       0.0885      0.073      1.205      0.231      -0.057       0.234
BTC            0.6532      0.088      7.408      0.000       0.478       0.828
BNB_False     -0.0002      0.000     -0.734      0.465      -0.001       0.000
BNB_True       0.0001      0.000      0.333      0.740      -0.001       0.001
==============================================================================
Omnibus:                       28.448   Durbin-Watson:                   2.235
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               78.449
Skew:                          -0.970   Prob(JB):                     9.23e-18
Kurtosis:                       6.930   Cond. No.                     7.32e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.32e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [39]:
xrp_model2

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    XRP   R-squared:                       0.675
Model:                            OLS   Adj. R-squared:                  0.653
Method:                 Least Squares   F-statistic:                     31.45
Date:                Sun, 05 Jun 2022   Prob (F-statistic):           3.33e-20
Time:                        19:44:28   Log-Likelihood:                 204.65
No. Observations:                  98   AIC:                            -395.3
Df Residuals:                      91   BIC:                            -377.2
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0091      0.015     -0.606      0.546      -0.039       0.021
NASDAQ         0.2143      0.192      1.119      0.266      -0.166       0.595
GOLD           0.0553      0.359      0.154      0.878      -0.658       0.769
OILPRICE      -0.0496      0.104     -0.476      0.635      -0.256       0.157
BTC            0.6308      0.111      5.668      0.000       0.410       0.852
XRP_False     -0.0002      0.000     -0.851      0.397      -0.001       0.000
XRP_True       0.0004      0.000      1.895      0.061   -2.05e-05       0.001
==============================================================================
Omnibus:                       57.644   Durbin-Watson:                   1.978
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              553.484
Skew:                           1.548   Prob(JB):                    6.49e-121
Kurtosis:                      14.223   Cond. No.                     5.84e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.84e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [40]:
ada_model2

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    ADA   R-squared:                       0.769
Model:                            OLS   Adj. R-squared:                  0.754
Method:                 Least Squares   F-statistic:                     50.45
Date:                Sun, 05 Jun 2022   Prob (F-statistic):           7.52e-27
Time:                        19:44:28   Log-Likelihood:                 207.74
No. Observations:                  98   AIC:                            -401.5
Df Residuals:                      91   BIC:                            -383.4
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0285      0.055      0.517      0.606      -0.081       0.138
NASDAQ         0.3546      0.193      1.835      0.070      -0.029       0.738
GOLD           0.0430      0.348      0.124      0.902      -0.648       0.734
OILPRICE       0.1946      0.100      1.944      0.055      -0.004       0.393
BTC            0.7022      0.112      6.287      0.000       0.480       0.924
ADA_False     -0.0006      0.001     -1.008      0.316      -0.002       0.001
ADA_True   -8.504e-05      0.001     -0.142      0.887      -0.001       0.001
==============================================================================
Omnibus:                       11.612   Durbin-Watson:                   2.039
Prob(Omnibus):                  0.003   Jarque-Bera (JB):               28.091
Skew:                           0.270   Prob(JB):                     7.94e-07
Kurtosis:                       5.567   Cond. No.                     7.86e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.86e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [41]:
trn_model2

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   TRON   R-squared:                       0.568
Model:                            OLS   Adj. R-squared:                  0.539
Method:                 Least Squares   F-statistic:                     19.93
Date:                Sun, 05 Jun 2022   Prob (F-statistic):           9.65e-15
Time:                        19:44:28   Log-Likelihood:                 194.41
No. Observations:                  98   AIC:                            -374.8
Df Residuals:                      91   BIC:                            -356.7
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0159      0.035      0.453      0.652      -0.054       0.086
NASDAQ         0.2442      0.218      1.122      0.265      -0.188       0.676
GOLD           0.2326      0.405      0.575      0.567      -0.571       1.037
OILPRICE       0.2160      0.115      1.884      0.063      -0.012       0.444
BTC            0.3447      0.128      2.690      0.008       0.090       0.599
TRON_False    -0.0005      0.000     -1.128      0.262      -0.002       0.000
TRON_True   9.685e-05      0.000      0.203      0.840      -0.001       0.001
==============================================================================
Omnibus:                       45.924   Durbin-Watson:                   2.586
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              164.314
Skew:                           1.542   Prob(JB):                     2.09e-36
Kurtosis:                       8.543   Cond. No.                     6.34e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.34e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [42]:
dog_model2

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   DOGE   R-squared:                       0.613
Model:                            OLS   Adj. R-squared:                  0.587
Method:                 Least Squares   F-statistic:                     23.99
Date:                Sun, 05 Jun 2022   Prob (F-statistic):           7.68e-17
Time:                        19:44:28   Log-Likelihood:                 186.77
No. Observations:                  98   AIC:                            -359.5
Df Residuals:                      91   BIC:                            -341.4
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0176      0.015     -1.154      0.252      -0.048       0.013
NASDAQ         0.2608      0.230      1.134      0.260      -0.196       0.718
GOLD          -0.1597      0.434     -0.368      0.714      -1.021       0.702
OILPRICE       0.0651      0.124      0.525      0.601      -0.181       0.311
BTC            0.4774      0.142      3.370      0.001       0.196       0.759
DOGE_False    -0.0002      0.000     -0.719      0.474      -0.001       0.000
DOGE_True      0.0006      0.000      2.645      0.010       0.000       0.001
==============================================================================
Omnibus:                       21.861   Durbin-Watson:                   2.515
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              155.916
Skew:                           0.086   Prob(JB):                     1.39e-34
Kurtosis:                       9.177   Cond. No.                     6.02e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.02e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [86]:
sbi_model2

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   SHIB   R-squared:                       0.726
Model:                            OLS   Adj. R-squared:                  0.708
Method:                 Least Squares   F-statistic:                     40.20
Date:                Sun, 05 Jun 2022   Prob (F-statistic):           1.53e-23
Time:                        19:52:06   Log-Likelihood:                 186.79
No. Observations:                  98   AIC:                            -359.6
Df Residuals:                      91   BIC:                            -341.5
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0137      0.009     -1.520      0.132      -0.032       0.004
NASDAQ         0.2245      0.232      0.969      0.335      -0.236       0.685
GOLD           0.3245      0.430      0.754      0.453      -0.530       1.179
OILPRICE       0.1217      0.124      0.981      0.329      -0.125       0.368
BTC            0.6456      0.139      4.641      0.000       0.369       0.922
SHIB_False    -0.0004      0.000     -2.207      0.030      -0.001   -4.35e-05
SHIB_True      0.0010      0.000      5.254      0.000       0.001       0.001
==============================================================================
Omnibus:                       20.073   Durbin-Watson:                   2.512
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               74.118
Skew:                          -0.493   Prob(JB):                     8.04e-17
Kurtosis:                       7.145   Cond. No.                     4.64e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.64e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""